.csv uzantılı dosyanın yüklemesinin gerçekleştirilmesi

In [ ]:
from langchain_community.document_loaders.csv_loader import CSVLoader


loader = CSVLoader(file_path='data/yemek_chatbot_veriseti.csv')
data = loader.load()

In [129]:
len(data)

1350

Veriyi parçalara ayırma (Chunking)

In [130]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000)
docs = text_splitter.split_documents(data)

print("Total",len(docs))

Total 1350


In [131]:
docs [7]

Document(metadata={'source': 'yemek_chatbot_veriseti.csv', 'row': 7}, page_content='text: Günaydın\nintent: selamlama')

OpenAI Generative AI Embeddings'i kullanarak Embedding oluşturma kısmı
- Text veriyi sayısal veriye çevirme

In [132]:
import streamlit as st
from langchain_community.document_loaders import CSVLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import Chroma
from langchain.embeddings import OpenAIEmbeddings
from langchain_openai import ChatOpenAI
from langchain.chains import create_retrieval_chain
from langchain.prompts import ChatPromptTemplate
from langchain.chains.combine_documents import create_stuff_documents_chain

In [133]:
from dotenv import load_dotenv
load_dotenv()

True

In [134]:
embeddings = OpenAIEmbeddings(model="text-embedding-3-large")

In [135]:
from langchain.schema import Document

In [136]:
unique_docs = list(set([doc.page_content for doc in docs]))
docs = [Document(page_content=content) for content in unique_docs]

In [137]:
vectorstore = Chroma.from_documents(
    documents=docs,
    embedding=embeddings,
    persist_directory="./chroma_db"
)

In [138]:
retriever = vectorstore.as_retriever(
    search_type="similarity", search_kwargs={"k": 5}
)

In [139]:
retieved_docs = retriever.invoke("tatlı önerisi")

In [140]:
len(retieved_docs)

5

In [142]:
print(retieved_docs[4].page_content)

text: Sağlıklı tatlı önerileri
intent: tatli


In [143]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(
    model="gpt-4o",
    temperature=0.3,
    max_tokens=500
)

In [144]:
system_prompt = (
    "Sen, yemek önerileri veren cana yakın bir asistansın. "
    "Görevin, kısa ama etkileyici cüevaplar bulup samimi bir dille cevaplamak "
    "Sadece yemekler üzerine konuş başka bir konu hakkında konuşma"
    "Yanıtların Türkçe olsun 🇹🇷 ve açıklamalarını emojilerle zenginleştir 📌"
    "\n\n"
    "{context}"
)

In [145]:
prompt = ChatPromptTemplate.from_messages(
    [
        ("system",system_prompt),
        ("human","{input}")
    ]
)

In [146]:
question_answer_chain = create_stuff_documents_chain(llm,prompt)

In [147]:
rag_chain = create_retrieval_chain(retriever,question_answer_chain)

In [125]:
response = rag_chain.invoke({"input":"tatlı önerisi"})

In [148]:
print(response)

{'input': 'tatlı önerisi', 'context': [Document(metadata={'row': 644, 'source': 'yemek_chatbot_veriseti.csv'}, page_content='text: Sağlıklı tatlı önerileri\nintent: tatli'), Document(metadata={'row': 719, 'source': 'yemek_chatbot_veriseti.csv'}, page_content='text: Sağlıklı tatlı önerileri\nintent: tatli'), Document(metadata={}, page_content='text: Sağlıklı tatlı önerileri\nintent: tatli'), Document(metadata={'row': 748, 'source': 'yemek_chatbot_veriseti.csv'}, page_content='text: Sağlıklı tatlı önerileri\nintent: tatli'), Document(metadata={}, page_content='text: Sağlıklı tatlı önerileri\nintent: tatli'), Document(metadata={'row': 601, 'source': 'yemek_chatbot_veriseti.csv'}, page_content='text: Sağlıklı tatlı önerileri\nintent: tatli'), Document(metadata={'row': 601, 'source': 'yemek_chatbot_veriseti.csv'}, page_content='text: Sağlıklı tatlı önerileri\nintent: tatli'), Document(metadata={'row': 654, 'source': 'yemek_chatbot_veriseti.csv'}, page_content='text: Sağlıklı tatlı önerileri

In [149]:
print(response["answer"])

Tabii! Chia tohumlu puding deneyebilirsin. Süt, chia tohumu ve balı karıştır, buzdolabında beklet. Üzerine taze meyve ekle, işte sağlıklı ve lezzetli bir tatlı! 🍓🥥
